In [1]:
import sys, os, git, time

current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(git_path+"/python-codes/")

from InitializeSimulation import InitializeSimulation
from Utilities import Utilities
from Outputs import Outputs
from MolecularDynamics import MolecularDynamics
from MonteCarlo import MonteCarlo

import numpy as np

In [2]:
self = MolecularDynamics(number_atoms=[50, 50],
                      epsilon=[0.1, 0.1], # kcal/mol
                      sigma=[2, 1], # A
                      atom_mass=[1, 1], # g/mol
                      atom_charge=[0.1,-0.1], # in elementary charge units
                      Lx=30, # A
                      Ly=30, # A
                      Lz=30, # A
                      minimization_steps = 50,
                      maximum_steps=500,
                      desired_temperature=300,
                      thermo = 10,
                      dump = 10,  
                      tau_temp = 100, # fs
                      #tau_press= 1000, # fs
                      time_step=1, # fs
                      seed=319817,
                      data_folder = "mdcode-output/",
                      )
self.run()
# run lammps for comparison
os.system("/home/simon/Softwares/lammps-2Aug2023/src/lmp_serial -in input.lammps")

step epot maxF
0 141616.186 5345485.986
5 856.952 36173.248
10 -0.368 14.793
15 -1.344 0.180
20 -1.511 0.202
25 -1.610 0.193
30 -1.674 0.231
35 -1.815 0.191
40 -1.913 0.226
45 -1.977 0.165
50 -1.999 0.085
 step     N  temp  epot  ekin press   vol
    0   100 299.97054616262244 -1.9907489036159403 88.52125789997515 148.18015040958107 27000.0
   10   100 295.98876436562125 -0.7695100948384294 87.34623475899423 159.16011006045377 27000.0
   20   100 298.0771577353297 -1.279773516207083 87.96251929239753 150.95415287209752 27000.0
   30   100 298.3387709804332 -1.3053958188088093 88.03972131718331 146.853146246287 27000.0
   40   100 298.39133045035004 -1.2719609178086433 88.05523160794723 145.22974671542184 27000.0
   50   100 297.40569082783094 -0.9284264427947874 87.764369520527 149.13343630599783 27000.0
   60   100 296.4944503598689 -0.5328846620236388 87.49546261114786 153.6628697048578 27000.0
   70   100 297.42046963318967 -0.7324763224830297 87.76873074351155 150.54010658908425 27

0

In [3]:
from itertools import combinations
from scipy.special import erfc # complementary error function : ercf = 1 - erf
import numpy as np

### pot $r$ Ewald

In [4]:
# electrostatic potential due to a point charge qi surounded by a Gaussian with net charge -qi 
# pot_short-range = qi/r - qi/r erf( sqrt(alpha)r ) = qi/r erfc( sqrt(alpha)r )
# total contribution:
# U_short-range = 0.5 * sum_{i \ne j} q_i q_j erfc( sqrt(alpha r_ij) ) / r_ij
# tofix : careful with double count ?
pot = 0
kappa = 6
for position_i, charge_i in zip(self.atoms_positions, self.atoms_charge):
    positions_j = self.atoms_positions
    charges_j = self.atoms_charge
    rij_xyz = (np.remainder(position_i - positions_j
        + self.box_size/2., self.box_size) - self.box_size/2.)
    norm_rij = np.linalg.norm(rij_xyz, axis=1)
    vij = charge_i * charges_j[norm_rij>0] \
          * erfc ( kappa * norm_rij[norm_rij>0] ) / norm_rij[norm_rij>0] # Screened Coulomb term
    pot += np.sum(vij)
print(pot)

3.2494945005328153e-10


### pot $k$ Ewald

In [5]:
from itertools import product

In [6]:
twopi = 2.0*np.pi
twopi_sq = twopi**2

# first call

kappa = 6
nk = 8
b = 1.0 / 4.0 / kappa / kappa
k_sq_max = nk**2 
kfac = np.zeros(k_sq_max+1,dtype=np.float_)

for kx,ky,kz in product(range(nk+1),repeat=3):
    k_sq = kx**2 + ky**2 + kz**2
    if k_sq <= k_sq_max and k_sq>0: # Test to ensure within range
        kr_sq      = twopi_sq * k_sq           # k**2 in real units
        kfac[k_sq] = twopi * np.exp ( -b * kr_sq ) / kr_sq # Stored expression for later use

In [7]:
# other call
eikx = np.zeros((self.total_number_atoms,nk+1),  dtype=np.complex_) # omits negative k indices
eiky = np.zeros((self.total_number_atoms,2*nk+1),dtype=np.complex_) # includes negative k indices
eikz = np.zeros((self.total_number_atoms,2*nk+1),dtype=np.complex_) # includes negative k indices

# Calculate kx, ky, kz = 0, 1 explicitly
eikx[:,   0] = 1.0 + 0.0j
eiky[:,nk+0] = 1.0 + 0.0j
eikz[:,nk+0] = 1.0 + 0.0j

eikx[:,   1] = np.cos(twopi*self.atoms_positions[:,0]) + np.sin(twopi*self.atoms_positions[:,0])*1j
eiky[:,nk+1] = np.cos(twopi*self.atoms_positions[:,1]) + np.sin(twopi*self.atoms_positions[:,1])*1j
eikz[:,nk+1] = np.cos(twopi*self.atoms_positions[:,2]) + np.sin(twopi*self.atoms_positions[:,2])*1j

# Calculate remaining positive kx, ky and kz by recurrence
for k in range(2,nk+1):
    eikx[:,   k] = eikx[:,   k-1] * eikx[:,   1]
    eiky[:,nk+k] = eiky[:,nk+k-1] * eiky[:,nk+1]
    eikz[:,nk+k] = eikz[:,nk+k-1] * eikz[:,nk+1]

# Negative k values are complex conjugates of positive ones
# We do not need negative values of kx
eiky[:,0:nk] = np.conj ( eiky[:,2*nk:nk:-1] )
eikz[:,0:nk] = np.conj ( eikz[:,2*nk:nk:-1] )


pot = 0.0

for kx in range(nk+1): # Outer loop over non-negative kx

    factor = 1.0 if kx==0 else 2.0 # Accounts for skipping negative kx

    for ky,kz in product(range(-nk,nk+1),repeat=2):  # Double loop over ky, kz vector components

        k_sq = kx**2 + ky**2 + kz**2

        if k_sq <= k_sq_max and k_sq > 0: # Test to ensure within range

            term = np.sum ( self.atoms_charge * eikx[:,kx] * eiky[:,nk+ky] * eikz[:,nk+kz] ) # Sum over all ions
            pot  = pot + factor * kfac[k_sq] * np.real ( np.conj(term)*term )

# Subtract self part of k-space sum
pot = pot - kappa * np.sum ( self.atoms_charge**2 ) / np.sqrt(np.pi)

print(pot)

-0.8549626132409531


In [8]:
self.atoms_charge[:]

array([ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,
        0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,
        0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,
        0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,
        0.1,  0.1,  0.1,  0.1,  0.1,  0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
       -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
       -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
       -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
       -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
       -0.1])

In [9]:
eikx

array([[ 1.        +0.j        ,  0.18859214+0.9820555j ,
        -0.92886601+0.3704159j , -0.53894581-0.84234044j,
         0.72558412-0.68813348j,  0.81262474+0.5827873j ,
        -0.41907483+0.9079517j , -0.97069318-0.24032219j,
         0.05294462-0.99859745j],
       [ 1.        +0.j        ,  0.33730942-0.94139384j,
        -0.77244471-0.63508202j, -0.85841518+0.51295554j,
         0.19334166+0.98113149j,  0.9888471 +0.14893425j,
         0.47375323-0.88065764j, -0.66924425-0.74304249j,
        -0.92523801+0.37938718j],
       [ 1.        +0.j        , -0.6069568 -0.79473483j,
        -0.2632069 +0.96473941j,  0.92646723-0.37637545j,
        -0.86144426-0.50785213j,  0.11925167+0.99286406j,
         0.71668304-0.69739904j, -0.98924295-0.14628188j,
         0.48417242+0.87497261j],
       [ 1.        +0.j        , -0.96347477-0.26779913j,
         0.85656726+0.5160354j , -0.68708711-0.72657505j,
         0.46741493+0.88403806j, -0.21359787-0.97692167j,
        -0.05582261+0.998440

In [10]:







# Calculate remaining positive kx, ky and kz by recurrence
for k in range(2,nk+1):
    eikx[:,   k] = eikx[:,   k-1] * eikx[:,   1]
    eiky[:,nk+k] = eiky[:,nk+k-1] * eiky[:,nk+1]
    eikz[:,nk+k] = eikz[:,nk+k-1] * eikz[:,nk+1]

# Negative k values are complex conjugates of positive ones
# We do not need negative values of kx
eiky[:,0:nk] = np.conj ( eiky[:,2*nk:nk:-1] )
eikz[:,0:nk] = np.conj ( eikz[:,2*nk:nk:-1] )

pot = 0.0

for kx in range(nk+1): # Outer loop over non-negative kx

    factor = 1.0 if kx==0 else 2.0 # Accounts for skipping negative kx

    for ky,kz in product(range(-nk,nk+1),repeat=2):  # Double loop over ky, kz vector components

        k_sq = kx**2 + ky**2 + kz**2

        if k_sq <= k_sq_max and k_sq > 0: # Test to ensure within range

            term = np.sum ( q[:] * eikx[:,kx] * eiky[:,nk+ky] * eikz[:,nk+kz] ) # Sum over all ions
            pot  = pot + factor * kfac[k_sq] * np.real ( np.conj(term)*term )

# Subtract self part of k-space sum
pot = pot - kappa * np.sum ( q**2 ) / np.sqrt(np.pi)

NameError: name 'q' is not defined

64

64